# Building structures

In the following we show examples on how to generate some complex structures like surfaces, supercells, heterostructures.
Basically, we rely on already well established libraries (ASE, Pymatgen) to build up the structure and attach some properties, and then we generate the `StructureDataMutable`.

## Surfaces 

The first examples is a simple Al(111) surface with an adsorbed H atom on top. We follow the [ASE example](https://wiki.fysik.dtu.dk/ase/ase/build/surface.html). Moreover, we want to have our adsorbed H with +1 charge.

In [24]:
from aiida import load_profile
load_profile()

from aiida_atomistic import StructureData, StructureDataMutable

from ase.build import fcc111, add_adsorbate

slab = fcc111('Al', size=(2,2,3))
add_adsorbate(slab, 'H', 1.5, 'ontop')

slab.set_initial_charges([0]*(len(slab)-1)+[1])

slab.center(vacuum=10.0, axis=2)

Then, we can transform it into our `StructureDataMutable`:

In [25]:
mutable_structure = StructureDataMutable.from_ase(slab)
mutable_structure.properties.sites

[<SiteImmutable: kind name 'Al0' @ 1.4318912319027588,0.8267027881893224,10.0>,
 <SiteImmutable: kind name 'Al0' @ 4.295673695708277,0.8267027881893224,10.0>,
 <SiteImmutable: kind name 'Al0' @ 2.8637824638055176,3.3068111527572897,10.0>,
 <SiteImmutable: kind name 'Al0' @ 5.727564927611035,3.3068111527572897,10.0>,
 <SiteImmutable: kind name 'Al0' @ -1.5983186326405526e-17,1.6534055763786448,12.338268590217984>,
 <SiteImmutable: kind name 'Al0' @ 2.8637824638055176,1.6534055763786448,12.338268590217984>,
 <SiteImmutable: kind name 'Al0' @ 1.4318912319027586,4.133513940946612,12.338268590217984>,
 <SiteImmutable: kind name 'Al0' @ 4.295673695708277,4.133513940946612,12.338268590217984>,
 <SiteImmutable: kind name 'Al0' @ 0.0,0.0,14.676537180435968>,
 <SiteImmutable: kind name 'Al0' @ 2.8637824638055176,0.0,14.676537180435968>,
 <SiteImmutable: kind name 'Al0' @ 1.4318912319027588,2.4801083645679673,14.676537180435968>,
 <SiteImmutable: kind name 'Al0' @ 4.295673695708277,2.480108364567

We can then verify that indeed the periodic boundary conditions (PBC) are the expected ones:

In [26]:
mutable_structure.properties.pbc

[True, True, False]

and that indeed the last sites in the structure (the H) is positively charged:

In [28]:
mutable_structure.properties.charges

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]

:::{note}
We could have added the charge also via the `StructureDataMutable.set_charges` method. The only advantage of doing it before the conversion is that the charge will be then taken into account for the automatic detection of kinds when the `from_ase` method is invoked.
:::

## Supercells



In [40]:
from pymatgen.core.structure import Structure

structure = Structure.from_file('../examples/structure/data/Si.cif') 
structure.make_supercell(3)

Structure Summary
Lattice
    abc : 11.600923949999999 11.600923949999999 11.60092395
 angles : 59.999999999999986 59.999999999999986 59.999999999999986
 volume : 1103.9839041177904
      A : 10.046694848071313 0.0 5.800461975000001
      B : 3.3488982826904383 9.472114740777569 5.800461975000001
      C : 0.0 0.0 11.60092395
    pbc : True True True
PeriodicSite: Si0 (Si) (3.349, 2.368, 5.8) [0.25, 0.25, 0.25]
PeriodicSite: Si0 (Si) (3.349, 2.368, 9.667) [0.25, 0.25, 0.5833]
PeriodicSite: Si0 (Si) (3.349, 2.368, 13.53) [0.25, 0.25, 0.9167]
PeriodicSite: Si0 (Si) (4.465, 5.525, 7.734) [0.25, 0.5833, 0.25]
PeriodicSite: Si0 (Si) (4.465, 5.525, 11.6) [0.25, 0.5833, 0.5833]
PeriodicSite: Si0 (Si) (4.465, 5.525, 15.47) [0.25, 0.5833, 0.9167]
PeriodicSite: Si0 (Si) (5.581, 8.683, 9.667) [0.25, 0.9167, 0.25]
PeriodicSite: Si0 (Si) (5.581, 8.683, 13.53) [0.25, 0.9167, 0.5833]
PeriodicSite: Si0 (Si) (5.581, 8.683, 17.4) [0.25, 0.9167, 0.9167]
PeriodicSite: Si0 (Si) (6.698, 2.368, 7.734) [0.583

In [41]:
mutable_structure = StructureDataMutable.from_pymatgen(structure)
mutable_structure.properties.cell

[[10.046694848071313, 0.0, 5.800461975000001],
 [3.3488982826904383, 9.472114740777569, 5.800461975000001],
 [0.0, 0.0, 11.60092395]]

We can notice that actually the `kinds` now are collapsed into only one, because when we read the Pymatgen structure, an automatic detection of kinds runs and only one if found, i.e. there are no properties which allow a particular distinction:

In [42]:
set(mutable_structure.properties.kinds)

{'Si0'}

we can actually untrigger this detection by setting the `detect_kinds` input parameter to `False`, when we initialise the `StructureDataMutable` from Pymatgen:

In [44]:
mutable_structure = StructureDataMutable.from_pymatgen(structure,detect_kinds=False)
set(mutable_structure.properties.kinds)

{'Si0', 'Si1'}

### Supercell and properties

Usually, we want the properties to be propagated in to atoms in a supercell, when we generate it from a unit cell. This can be ensured, once again, using the ASE/Pymatgen libraries:

In [58]:
from ase.build import bulk, make_supercell
atoms = bulk('NaCl', 'rocksalt', a=5.64)
atoms.set_initial_charges([1, -1] * (len(atoms) // 2))

In [61]:
supercell = make_supercell(atoms, [[2, 0, 0], [0, 2, 0], [0, 0, 2]])

In [62]:
supercell.get_initial_charges()

array([ 1., -1.,  1., -1.,  1., -1.,  1., -1.,  1., -1.,  1., -1.,  1.,
       -1.,  1., -1.])

In [63]:
mutable_structure = StructureDataMutable.from_ase(supercell)

In [64]:
mutable_structure.properties.charges

[1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 -1.0]

Of course, if we want to add a property which is only supported in the atomistic package, e.g. hubbard, we can only attach it once the `StructureDataMutable` of the ASE/Pymatgen supercell is generated.

In [67]:
set(mutable_structure.properties.kinds)

{'Cl1', 'Na0'}

In [70]:
mutable_structure.initialize_onsites_hubbard("Na0", '3d', 4, 'U', use_kinds=True)
mutable_structure.properties.hubbard.parameters

[HubbardParameters(atom_index=0, atom_manifold='3d', neighbour_index=0, neighbour_manifold='3d', translation=(0, 0, 0), value=4.0, hubbard_type='U')]

please note that this is just a dummy example where we put a random hubbard U on sodium. For more details on how to set up hubbard parameters, please refer to the [How to run DFT+U (+V)](03-tutorial_qe.ipynb#how-to-run-DFT+U-+V) section.